In [12]:
import unidecode
import requests
import pandas as pd
import numpy as np
import re
from dateutil.parser import parse
from random import randint
from time import sleep
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from __future__ import division
import collections
from collections import Counter
from sklearn import grid_search, decomposition
import pickle
import json
from pymongo import MongoClient
client = MongoClient()
# To Plot matplotlib figures inline on the notebook
%matplotlib inline
from textblob import TextBlob
from textblob.np_extractors import ConllExtractor, FastNPExtractor
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from bs4 import BeautifulSoup
def split_into_lemmas(message):
    words = []
    message = unidecode.unidecode(message)
    message = TextBlob(message).lower()
    for i in message.words:
        if len(i)>1:
            words.append(i)
    #words2 = blobextract(message)
    #result = words2 + words
    # for each word, take its "base form" = lemma 
    return words

def unescape(text):
    def fixup(m):
        text = m.group(0)
        if text[:2] == "&#":
            # character reference
            try:
                if text[:3] == "&#x":
                    return unichr(int(text[3:-1], 16))
                else:
                    return unichr(int(text[2:-1]))
            except ValueError:
                pass
        else:
            # named entity
            try:
                text = unichr(htmlentitydefs.name2codepoint[text[1:-1]])
            except KeyError:
                pass
        return text # leave as is
    return re.sub("&#?\w+;", fixup, text)
def makesoup(link):
    response = requests.get(link)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    return soup

In [15]:
#pickle.dump(nmf, open( "nmf_model.p", "wb" ) )
#pickle.dump(tfidf_vectorizer, open('tfidf.p', 'wb'))
#pickle.dump(topics, open('topics.p', 'wb'))
nmf = pickle.load( open( "nmf_model.p", "rb" ) )
tfidf_vectorizer = pickle.load( open( "tfidf.p", "rb" ) )
topics = pickle.load( open( "topics.p", "rb" ) )
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [55]:
link = 'http://www.nytimes.com/2016/11/05/world/africa/south-sudan-united-nations.html'
soup = makesoup(link)
title = unescape(soup.find('h1', class_ = 'headline').text)
text = unescape(' '.join([j.text for j in soup.findAll('p', class_ = 'story-body-text story-content')]))

In [52]:

def get_sentences(text):
    highlited = ''
    #transform document into existing nytimes tfidf space
    mew = tfidf_vectorizer.transform([text])
    #transform vectorized document into topic space
    mu = nmf.transform(mew)[0]
    b = mu.argsort()[::-1]
    num = len(mu[mu>=0.008])
    topic2word = [{} for _ in range(num)]
    topicw = np.zeros(num)
    z = 0
    #make dictionary of words to weight for each top topic
    for ind, i in enumerate(b[:num]):
        if i == 0:
            z = 1
            print ('general interest')
        else:
            print (topics[i], mu[i])
            weights = nmf.components_[i][nmf.components_[i]>0]
            words = np.array(tfidf_feature_names)[nmf.components_[i]>0]
            dic = zip(words, weights)
            topic2word[ind-z] = dict((x, y) for x, y in dic)
            topicw[ind-z] = mu[i]
    topicw = topicw/np.max(topicw)
    
    n_top_sen = 5
    document = unidecode.unidecode(text)
    blob = TextBlob(text).lower()
    sentence_score = []

    for j in blob.sentences:
        score = 0
        for i in set(j.words):
            for ind, topic in enumerate(topic2word):
                if i in topic.keys():
                    score += topic[i]#*topicw[ind]
        if len(j.words) >= 10:
            #print j
            #print score/len(j.words)
            sentence_score.append(score/len(j.words))
        else:
            #print j
            #print 'too short!!'
            sentence_score.append(0)
    sentence_score = np.array(sentence_score)
    indlist = sentence_score.argsort()[:-n_top_sen - 1:-1]
    for ind, sent in enumerate(TextBlob(text).sentences):
        
        if ind == indlist[0]:
            new = ' <p class="bg-warning">'+str(sent)+'</p>'
        elif ind in indlist[1:]:
            new = ' <p class="bg-warning">'+str(sent)+'</p>'       
        else: 
            new = ' '+str(sent)
        highlited += new
        
    #sentence_list = '\n'.join([str(TextBlob(text).sentences[i] for i in sentence_score.argsort()[:-n_top_sen - 1:-1]] )
    return highlited

In [17]:
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [18]:
b = mu.argsort()[::-1]
num = len(mu[mu>=0.008])
topic2word = [{} for _ in range(num)]
topicw = np.zeros(num)
z = 0
#make dictionary of words to weight for each top topic
for ind, i in enumerate(b[:num]):
    if i == 0:
        z = 1
        print ('general interest')
    else:
        print (topics[i], mu[i])
        weights = nmf.components_[i][nmf.components_[i]>0]
        words = np.array(tfidf_feature_names)[nmf.components_[i]>0]
        dic = zip(words, weights)
        topic2word[ind-z] = dict((x, y) for x, y in dic)
        topicw[ind-z] = mu[i]
topicw = topicw/np.max(topicw)


(u'party-republican-republicans-voters-democrats', 0.04323008628693166)
(u'trump-donald-republican-campaign-mr', 0.039219979535119194)
(u'clinton-mrs-campaign-hillary-voters', 0.034428165538774633)
(u'obama-president-white-administration-house', 0.017386952756056555)
general interest


In [20]:
n_top_sen = 5
document = unidecode.unidecode(text)
blob = TextBlob(text).lower()
sentence_score = []

for j in blob.sentences:
    score = 0
    for i in set(j.words):
        for ind, topic in enumerate(topic2word):
            if i in topic.keys():
                score += topic[i]#*topicw[ind]
    if len(j.words) >= 10:
        #print j
        #print score/len(j.words)
        sentence_score.append(score/len(j.words))
    else:
        #print j
        #print 'too short!!'
        sentence_score.append(0)


In [38]:
sentence_score = np.array(sentence_score)
sentence_list = [TextBlob(text).sentences[i] for i in sentence_score.argsort()[:-n_top_sen - 1:-1]] 


In [54]:
sentence_score.argsort()[:-n_top_sen - 1:-1]

array([33, 20,  2, 13, 30])

In [32]:
len(TextBlob(text).sentences)

42

In [56]:
get_sentences(text)

(u'islamic-syria-state-united-government', 0.036848054640386638)
(u'obama-president-white-administration-house', 0.014281542737056892)
(u'korea-north-nuclear-korean-south', 0.012932001561166468)
general interest


' <p class="bg-warning">UNITED NATIONS \xe2\x80\x94 An American diplomat who helped shape the Obama administration\xe2\x80\x99s efforts to steer South Sudan to independence in 2011 says he is appalled by the actions of its leaders, including those he personally knows.</p> <p class="bg-warning">\xe2\x80\x9cI\xe2\x80\x99m horrified and disgusted by it,\xe2\x80\x9d said the diplomat, David Pressman, an alternate American representative to the United Nations.</p> \xe2\x80\x9cHow can you not be?\xe2\x80\x9d Mr. Pressman leaves his post next week to become partner in the law firm of Boies, Schiller & Flexner. \xe2\x80\x9cThe leadership of this country were handed an incredible opportunity,\xe2\x80\x9d he said in an interview, along with what he called \xe2\x80\x9chuge amounts of assistance from the United States and from U.S. taxpayers.\xe2\x80\x9d \xe2\x80\x9cThe choices that have been made are the choices that have led to a fundamental shaking of the nascent foundations of the country, has